In [3]:
# !git clone https://github.com/ultralytics/yolov5.git
# %cd yolov5
# !pip install -U -r requirements.txt
!pip install ultralytics


In [ ]:

#___DATA_DOWNLOAD___#
#!curl -O /content/kitti/ https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
#!curl -O /content/kitti/ https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip

In [ ]:
#!python -m unzip data_object_image_2.zip -d /content/kitti/
#!python -m unzip data_object_label_2.zip -d /content/kitti/

In [ ]:
#converting the dataset to yolo accepable dataset
import os
import cv2

# output directories
output_image_dir = 'content/kitti/yolo_images'
output_label_dir = 'content/kitti/yolo_labels'
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

class_map = {
    "pedestrian": 0,
    "person sitting": 1,
    "dont care": 2,
    "misc": 3,
    "car": 4,
    "cyclist": 5,
    "truck": 6,
    "tram": 7,
    "van": 8
}

def kitti_to_yolo(kitti_image_dir, kitti_label_dir):
    for image_file in os.listdir(kitti_image_dir):
        image_path = os.path.join(kitti_image_dir, image_file)
        # image_path = kitti_image_dir + f'/{image_file}'
        # print(f"img path: {image_path}")
        image = cv2.imread(image_path)
        height, width, _ = image.shape

        label_file = os.path.join(kitti_label_dir, image_file.replace(".png", ".txt"))
        if not os.path.exists(label_file):
            continue

        yolo_labels = []
        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_name = parts[0].lower()
                if class_name not in class_map:
                    continue  
                class_id = class_map[class_name]


                xmin = float(parts[4])
                ymin = float(parts[5])
                xmax = float(parts[6])
                ymax = float(parts[7])


                center_x = ((xmin + xmax) / 2) / width
                center_y = ((ymin + ymax) / 2) / height
                box_w = (xmax - xmin) / width
                box_h = (ymax - ymin) / height

                yolo_labels.append(f"{class_id} {center_x:.6f} {center_y:.6f} {box_w:.6f} {box_h:.6f}")


        yolo_label_path = os.path.join(output_label_dir, image_file.replace(".png", ".txt"))
        # print(f"yolo label: {yolo_label_path}")
        with open(yolo_label_path, 'w') as out_file:
            out_file.write("\n".join(yolo_labels))

        # print(f"out file path: {os.path.join(output_image_dir, image_file)}")
        cv2.imwrite(os.path.join(output_image_dir, image_file), image)


kitti_image_dir = r'content/kitti/training/image_2'
kitti_label_dir = r'content/kitti/training/label_2'


kitti_to_yolo(kitti_image_dir, kitti_label_dir)


In [ ]:
#spliting dataset into training and val
import os
import shutil
import random

image_dir = r'content/kitti/yolo_images'
label_dir = r'content/kitti/yolo_labels'

train_img_dir = r'content/kitti/images/train'
val_img_dir = r'content/kitti/images/val'
train_lbl_dir = r'content/kitti/labels/train'
val_lbl_dir = r'content/kitti/labels/val'


os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)


image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
random.shuffle(image_files)

split_idx = int(len(image_files) * 0.8)
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

def move_files(file_list, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for f in file_list:
        shutil.copy(os.path.join(src_img_dir, f), os.path.join(dst_img_dir, f))
        label_file = f.replace('.png', '.txt')
        if os.path.exists(os.path.join(src_lbl_dir, label_file)):
            shutil.copy(os.path.join(src_lbl_dir, label_file), os.path.join(dst_lbl_dir, label_file))

move_files(train_files, image_dir, label_dir, train_img_dir, train_lbl_dir)
move_files(val_files, image_dir, label_dir, val_img_dir, val_lbl_dir)


In [ ]:
#creating data.yaml file for yolo model
dataset_yaml = """
path: /content/kitti
train: images/train
val: images/val
nc: 9
names: ['pedestrian', 'person sitting', 'dont care', 'misc', 'car', 'cyclist', 'truck', 'tram', 'van']
"""
with open("content/kitti/data.yaml", "w") as f:
    f.write(dataset_yaml)

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:

#!python train.py --img 640 --batch 16 --epochs 50 --data ../kitti/data.yaml --weights yolov5s.pt --cache
#traing yolo model on dataset

from ultralytics import YOLO

model = YOLO('yolov8n.pt')  


# model.to('cuda')

model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=8,
    optimizer='AdamW',  
    lr0=0.001,
    lrf=0.0001,
    weight_decay=0.0005,
    momentum=0.937,
    device=0, 
    patience=10,
    box=7.5,
    cls=0.3,          
    fliplr=0.5,        
    mosaic=1.0,
    mixup=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4
)

New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.124  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=100, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning C:\Users\dhruv\OneDrive\Desktop\Anuprash Project\content\kitti\labels\train.cache... 6121 images, 49 backgrounds, 0 corrupt: 100%|██████████| 6121/6121 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 506.5149.6 MB/s, size: 774.0 KB)


val: Scanning C:\Users\dhruv\OneDrive\Desktop\Anuprash Project\content\kitti\labels\val.cache... 1634 images, 52 backgrounds, 0 corrupt: 100%|██████████| 1634/1634 [00:00<?, ?it/s]


Plotting labels to runs\detect\train27\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train27
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1634       8338      0.883      0.808       0.86      0.645
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 90, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



100 epochs completed in 3.573 hours.
Optimizer stripped from runs\detect\train27\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train27\weights\best.pt, 6.2MB

Validating runs\detect\train27\weights\best.pt...
Ultralytics 8.3.124  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 103/103 [00:16<00:00,  6.19it/s]


                   all       1634       8338      0.878      0.809      0.859      0.646
            pedestrian        378        908      0.905      0.648      0.777       0.47
                  misc        167        201      0.869      0.746      0.835       0.61
                   Car       1346       5930      0.851       0.89      0.874      0.709
               Cyclist        214        311      0.757      0.701      0.692       0.45
                 truck        247        251      0.959      0.892      0.951      0.788
                  tram         84        137      0.906       0.92      0.962      0.749
                   van        460        600      0.898      0.866      0.923      0.743
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\train27


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C0C9E96180>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [11]:
#!pip uninstall torch
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [9]:
#Check if tourch is based on cpu or gpu
import torch
import os

print(os.environ.get("CUDA_VISIBLE_DEVICES"))
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device() if torch.cuda.device_count() > 0 else "None")
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else "N/A")

import platform
print(platform.architecture())



0
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3050 Laptop GPU
('64bit', 'WindowsPE')


In [ ]:
from ultralytics import YOLO
import os
import glob
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

model_path = r'..\train27\weights\best.pt'  # path to your trained model
data_yaml = 'data.yaml' 
names = ['pedestrian', 'person sitting', 'dont care', 'misc', 'Car', 'Cyclist', 'truck', 'tram', 'van']
conf_threshold = 0.0  

model = YOLO(model_path)


metrics = model.val(data=data_yaml, save_conf=True, conf=conf_threshold, device='cuda')


conf_matrix_path = os.path.join(metrics.save_dir, 'confusion_matrix_normalized.png')


matrix = metrics.confusion_matrix.matrix  


plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=names, yticklabels=names)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("YOLOv8 Confusion Matrix on Validation Set")
plt.tight_layout()
plt.show()

Ultralytics 8.3.129  Python-3.12.10 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1685.0403.2 MB/s, size: 822.3 KB)


val: Scanning C:\Users\dhruv\OneDrive\Desktop\Anuprash Project\content\kitti\labels\val.cache... 1634 images, 52 backgrounds, 0 corrupt: 100%|██████████| 1634/1634 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 103/103 [01:25<00:00,  1.20it/s]


                   all       1634       8338      0.878      0.809      0.858      0.642
            pedestrian        378        908      0.903      0.648      0.775      0.465
                  misc        167        201      0.874      0.746      0.834      0.607
                   Car       1346       5930      0.851      0.891      0.873      0.704
               Cyclist        214        311      0.757      0.701      0.689      0.443
                 truck        247        251      0.959      0.892      0.951      0.789
                  tram         84        137        0.9       0.92      0.961      0.745
                   van        460        600      0.899      0.865      0.922      0.741
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 32.7ms postprocess per image
Results saved to runs\detect\val22


<Figure size 800x600 with 2 Axes>

In [1]:
from ultralytics import YOLO

model = YOLO(r"yolov8n.pt")

In [ ]:
model.tune(
    data='data.yaml',
    epochs=20,
    imgsz=640,
    iterations=5,
    optimizer='AdamW',
    space={
        'lr0': (1e-5, 1e-1),
        'lrf': (0.01, 1.0),
        'momentum': (0.6, 0.98),
        'weight_decay': (0.0, 0.001),
        'warmup_epochs': (0.0, 5.0),
        'warmup_momentum': (0.0, 0.95)
    },
    plots=True,
    save=True,
    val=True,
    patience=3
)

Tuner: Initialized Tuner instance with 'tune_dir=runs\detect\tune9'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/5 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8}
Saved runs\detect\tune9\tune_scatter_plots.png
Saved runs\detect\tune9\tune_fitness.png

Tuner: 1/5 iterations complete  (2090.67s)
Tuner: Results saved to runs\detect\tune9
Tuner: Best fitness=0.51991 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.80666, 'metrics/recall(B)': 0.65488, 'metrics/mAP50(B)': 0.73208, 'metrics/mAP50-95(B)': 0.49633, 'val/box_loss': 0.97508, 'val/cls_loss': 0.69287, 'val/dfl_loss': 1.00852, 'fitness': 0.51991}
Tuner: Best fitness model is runs\detect\train22
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs\detect\tune9\best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
w

In [10]:

"""
# YOLOv8 Model Evaluation 
"""

# !pip install ultralytics pyyaml matplotlib seaborn

import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display, HTML
import torch

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)


def load_data_config(data_yaml_path):
    with open(data_yaml_path, 'r') as file:
        data_config = yaml.safe_load(file)
    return data_config


def evaluate_model(model_path, data_yaml_path, batch_size=8, imgsz=640, device='cpu'):

    model = YOLO(model_path)
    
    results = model.val(
        data=data_yaml_path,
        batch=batch_size,
        imgsz=imgsz,
        device=device,
        split='val',
        plots=True,
        save_json=True
    )
    
    return results


def visualize_metrics(results, class_names):
    
    metrics = results.results_dict
    
    summary_data = {
        'Metric': ['mAP50', 'mAP50-95', 'Precision', 'Recall', 'F1 Score'],
        'Value': [
            metrics.get('metrics/mAP50(B)', 0),
            metrics.get('metrics/mAP50-95(B)', 0),
            metrics.get('metrics/precision(B)', 0),
            metrics.get('metrics/recall(B)', 0),
            2 * (metrics.get('metrics/precision(B)', 0) * metrics.get('metrics/recall(B)', 0)) / 
            (metrics.get('metrics/precision(B)', 0) + metrics.get('metrics/recall(B)', 0) + 1e-16)
        ]
    }
    df_summary = pd.DataFrame(summary_data)
    
    
    display(HTML("<h3>Evaluation Metrics Summary</h3>"))
    display(df_summary.style.background_gradient(cmap='Blues'))
    
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Metric', y='Value', data=df_summary, palette='viridis')
    plt.title('Model Evaluation Metrics', fontsize=16)
    plt.ylim(0, 1)
    plt.show()
    
    
    if hasattr(results, 'names') and class_names:
        class_metrics = []
        for i, class_name in enumerate(class_names):
            if f'metrics/AP50(B)-{class_name}' in metrics:
                class_metrics.append({
                    'Class': class_name,
                    'mAP50': metrics[f'metrics/AP50(B)-{class_name}']
                })
        
        if class_metrics:
            class_metrics = pd.DataFrame(class_metrics).sort_values('mAP50', ascending=False)
            
            plt.figure(figsize=(12, 6))
            sns.barplot(x='mAP50', y='Class', data=class_metrics, palette='rocket')
            plt.title('mAP50 by Class', fontsize=16)
            plt.xlim(0, 1)
            plt.show()
            
            display(HTML("<h4>Per-Class mAP50</h4>"))
            display(class_metrics.style.background_gradient(cmap='Reds', subset=['mAP50']))


MODEL_PATH = '../runs/detect/train27/weights/best.pt'
DATA_YAML_PATH = './data.yaml'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
IMG_SIZE = 640

data_config = load_data_config(DATA_YAML_PATH)
class_names = data_config.get('names', [])

print(f"Dataset configuration loaded with {len(class_names)} classes:")
print(class_names)

print(f"\nEvaluating model on {DEVICE}...")
results = evaluate_model(
    model_path=MODEL_PATH,
    data_yaml_path=DATA_YAML_PATH,
    batch_size=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE
)

visualize_metrics(results, class_names)

print("\nDetailed results saved in:")
print(results.save_dir)

Dataset configuration loaded with 9 classes:
['pedestrian', 'person sitting', 'dont care', 'misc', 'Car', 'Cyclist', 'truck', 'tram', 'van']

Evaluating model on cuda...
Ultralytics 8.3.129  Python-3.12.10 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2257.0904.3 MB/s, size: 814.1 KB)


val: Scanning C:\Users\dhruv\OneDrive\Desktop\Anuprash Project\content\kitti\labels\val.cache... 1634 images, 52 backgrounds, 0 corrupt: 100%|██████████| 1634/1634 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 205/205 [00:15<00:00, 12.82it/s]


                   all       1634       8338      0.881      0.807      0.859      0.646
            pedestrian        378        908       0.91      0.649      0.778       0.47
                  misc        167        201      0.878      0.746      0.835       0.61
                   Car       1346       5930      0.852       0.89      0.874       0.71
               Cyclist        214        311      0.759      0.701       0.69      0.446
                 truck        247        251       0.96      0.892      0.951      0.793
                  tram         84        137      0.906      0.912      0.961      0.749
                   van        460        600      0.899      0.862      0.923      0.745
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Saving runs\detect\val23\predictions.json...
Results saved to runs\detect\val23


,Metric,Value
0,mAP50,0.858972
1,mAP50-95,0.646054
2,Precision,0.880671
3,Recall,0.807432
4,F1 Score,0.842463


C:\Users\dhruv\AppData\Local\Temp\ipykernel_31672\4090168805.py:65: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Metric', y='Value', data=df_summary, palette='viridis')


<Figure size 1000x600 with 1 Axes>


Detailed results saved in:
runs\detect\val23
